# 1. import Dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
import datetime
import numpy as np
import gradio as gr
import keras

## 2. Data-set loading and Pre-processing
Data-set link: https://mega.nz/file/zcdywLhI#fck4ufXy_o_Uiu0vGqh-cZiKHw5Xe_n4M2qWUWSheAI

In [ ]:
# Set the path to the dataset folders
train_data_dir = 'path_to_train'
test_data_dir = 'path_to_val'

In [ ]:
batch_size = 32
epochs = 10
# Set the input image dimensions
img_width, img_height = 150, 150

In [ ]:
# Create an ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,        # Normalize pixel values between 0 and 1
    shear_range=0.2,        # Apply random shear transformations
    zoom_range=0.2,         # Apply random zoom transformations
    horizontal_flip=True)  # Flip images horizontally


# Only normalize pixel values for testing
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


# Load and normalize testing data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

## 3. Model Building and Training

In [ ]:
# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                           input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Train the model
model.fit(train_generator, epochs=epochs, callbacks=[tensorboard_callback])

## 4. Deployment using Gradio

In [ ]:


model = keras.models.load_model('x_ray.keras')

# Define the function for image classification
def classify_image(img):
    # Set the input image dimensions
    img_width, img_height = 150, 150

    # Resize the image to match the model's input shape
    img = img.resize((img_width, img_height))

    # Convert the image to a numpy array
    img = np.array(img)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)

    # Get the prediction
    prediction = model.predict(img)

    return "NOT fractured" if prediction > 0.5 else "fractured"

# Create a Gradio interface
iface = gr.Interface(
    fn=classify_image,
    inputs=gr.inputs.Image(type="pil", label="Upload an X-ray image"),
    outputs="text",
    title="Bone Fracture Classification",
    description="Upload an X-ray image, and this model will classify it as fractured or not.",
)

# Start the Gradio interface
iface.launch()
